# Demystifying Decorators

## Some you may have seen
- @staticmethod
- @classmethod
- @property
- @app.route or similar in flask or other frameworks


## What's a decorator?
It's an @ symbol followed by some name sitting just above a function, method, or class definition.

## Well, yeah... but what is it *really*?
The whole thing has three parts:
- @ symbol
- some name
- function, method, or class definition

The key is that @ symbol. It's an operator just like +, -, *, /, etc., and it has two operands:

- A callable
- A function, method, or class definition

## What's it do with them?

It is syntactic sugar for the following pattern:

In [1]:
# say we have some callable thing:

def times_two(n):
    return 2 * n

In [2]:
# and some other callable thing:

def my_decorator(obj):
    print "Wrapping %s!" % obj.__name__
    return obj

In [3]:
# and we pass the first thing into the second one and assign the result back to the first one.

times_two = my_decorator(times_two)

Wrapping times_two!


**The @ operator shortens this pattern. It invokes its first operand (my_decorator) against its second operand (times_two), and it assigns the result back to the symbol for the second operand (times_two).**

## And that's it. You now know everything about decorators.

In [4]:
@my_decorator
def times_two(n):
    return 2 * n

Wrapping times_two!


In [5]:
times_two

<function __main__.times_two>

### And you're sure there's no  magic...?

In [6]:
def im_a_function_that_returns_forty_two():
    return 42

im_a_function_that_returns_forty_two

<function __main__.im_a_function_that_returns_forty_two>

In [7]:
def nope_youre_four(obj):
    return 4

In [8]:
@nope_youre_four
def im_a_function_that_returns_forty_two():
    return 42

In [9]:
im_a_function_that_returns_forty_two

4

In [10]:
type(im_a_function_that_returns_forty_two)

int

### Didn't you say it worked on class definitions?

In [11]:
@my_decorator
class B(object):
    pass

Wrapping B!


In [12]:
B

__main__.B

# Great! So what can we do with them?

### Here's our "my_decorator" function again. It just returned whatever you passed to it.

In [13]:
def my_decorator(n):
    print "Returning what you gave me!"
    return n  # just return n

### What if we create a function inside a function and return that inner function instead of what was passed in?

In [14]:
def multiplier_factory(n):
    
    def multiplier(x):
        return n * x
    
    return multiplier

# mul_four and mul_five are functions gotten out of multiplier_factory

mul_four = multiplier_factory(4)
mul_five = multiplier_factory(5)

In [15]:
mul_four(3)

12

In [16]:
mul_five(3)

15

### Notice that the functions we get out of multiplier_factory remember the values of n that were passed into it.
Side note: A function that remembers some local environment in which it was defined is called a **closure**.

## What does that have to do with decorators?

Say you want to know how long a function is taking, and you want to encapsulate the timing logic so you can apply it to several different functions.

In [17]:
# Here's a little function that we think might be taking a while.

import time

def sum_to_n(a):
    """Sums integers."""
    total = a
    for x in xrange(1, a):
        total += x
        time.sleep(1)
    return total

In [18]:
import functools
import time

# Here's how we encapsulate our timing logic.

def timeit(func):

    def inner(x):
        """This is a timed func!"""
        start = time.time()
        try:
            return func(x)
        finally:
            duration = time.time() - start
            print "%s took %s" % (func.__name__, duration)

    return inner

# Notice how similar in structure this is to multiplier_factory

### To profile sum_to_n, we just wrap it in a timeit call

In [19]:
sum_to_n = timeit(sum_to_n)

**The function we got out of timeit and assign back to sum_to_n is just like mul_four and mul_five. It remembers the original thing we passed into timeit, in this case our original sum_to_n function.**

In [20]:
sum_to_n(2)

sum_to_n took 1.00109100342


3

### Is this pattern familiar?
Yep, look's just like the first thing we did with times_two and my_decorator. So let's just do this:

In [21]:
import time

@timeit
def sum_to_n(a):
    """Sums integers."""
    total = a
    for x in xrange(1, a):
        total += x
        time.sleep(1)
    return total

In [22]:
sum_to_n(3)

sum_to_n took 2.00203204155


6

### But what's this?

In [23]:
sum_to_n

<function __main__.inner>

In [24]:
sum_to_n.__doc__

'This is a timed func!'

## Cool... but what about decorators that take arguments?

Okay, so you noticed that our decorators haven't had any parens at the end. That's because the @ operator takes care of invocation for us. Remember, the operator takes two operands: a callable as the first and some function, method, or class definition to apply it against as the second.

Since the @ operator will call the first operand for us, it doesn't seem like we can give it any arguments. What can we do? Let's take a clue from multiplier_factory and timeit.

In [25]:
# Say we want a decorator that prints a custom message for each thing it decorates.

# Since a global msg won't work, let's make a factory function that takes a msg as an
# argument. Then let's create a function similar in structure to timeit inside *that* function.
# Like with timeit, we can return a locally defined function as the value of an enclosing
# function.

def make_noisy(msg):
    
    def decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print msg
            return func(*args, **kwargs)
        return inner
    
    return decorator

# We have to get an instance of the decorator back out of that whole thing, so let's ask
# the factory for one:

noise_maker = make_noisy("Calling times_four!")

# And now we can use it:

@noise_maker
def times_four(n):
    return 4 * n

times_four(4)

Calling times_four!


16

### Can we skip that assignment step?

Sure, let's just call make_noisy right after the @. We don't have to give the callable we're getting out of it a name first.

In [26]:
@make_noisy("Calling times_five!")
def times_five(n):
    return 5 * n

times_five(5)

Calling times_five!


25

## Neat! Show me something practical I can do with it.

### Configurable Automatic Retries

In [27]:
import functools

def retry(num_retries=3):
    
    def decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            exception = None
            attempts = num_retries + 1
            for attempt in xrange(1, attempts + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as ex:
                    print "Attempt %s failed!" % attempt
                    exception = ex
            raise exception
        return inner

    return decorator

In [28]:
import random

@retry(num_retries=5)
def flaky():
    if random.random() < 0.5:
        raise Exception("Boom!")
    return "Success!"

In [29]:
flaky()

'Success!'

### Memoization

We could have a function automatically remember its return values for given arguments. This can be useful if the function is computationally expensive or backed by a call that takes a long time to complete. Say it's reading from disk or making a network call it really doesn't *have* to make every time it's called.

In [30]:
# Here's a long running call to a service. The results don't change for the same arguments,
# and we don't need to remember every result.

# Terribad implementation for calculating fibonacci numbers for a given index. (Big-O complexity of 2^n)
# 1 1 2 3 5 8 13 21...
def fib(n):
    if n < 2:
        return 1
    return fib(n - 1) + fib(n - 2)

### Let's keep the results from the last few calls around in case they're needed again soon.

In [31]:
import functools
from collections import deque

def naive_hash(args, kwargs):
    return hash(str(args) + str(kwargs))


def memoizer(cache_size=5):
    # local state
    cache = {}
    args_queue = deque()
    cache_size = cache_size

    # inner functions
    def check_evict():
        if len(args_queue) > cache_size:
            old_key = args_queue.popleft()
            del cache[old_key]
            
    def should_compute(key):
        return key not in cache

    def memoize(key, value):
        cache[key] = value
        args_queue.append(key)
        
    def value(key):
        return cache[key]

    # the function that @ will call for us
    def decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            key = naive_hash(args, kwargs)
            if should_compute(key):
                memoize(key, func(*args, **kwargs))
            check_evict()
            return value(key)
        return inner

    return decorator

In [32]:
@memoizer(cache_size=3)
def memfib(n):
    if n < 2:
        return 1
    return memfib(n - 1) + memfib(n - 2)

In [33]:
# This wouldn't complete in our lifetimes without memoization.
memfib(100)

573147844013817084101L

## Awesome, but wouldn't that be hard to test or change eviction policy?
Yep, and we already have a thing to capture local state and related functions: let's use a class.

In [34]:
import functools
from collections import deque


class Memoizer(object):
    def __init__(self, cache_size=5):
        self.cache = {}
        self.args_queue = deque()
        self.cache_size = cache_size

    def _check_evict(self):
        if len(self.args_queue) > self.cache_size:
            old_key = self.args_queue.popleft()
            del self.cache[old_key]
            
    def _should_compute(self, key):
        return key not in self.cache

    def _memoize(self, key, value):
        self.cache[key] = value
        self.args_queue.append(key)
        
    def _value(self, key):
        return self.cache[key]
        
    def __call__(self, func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            key = naive_hash(args, kwargs)
            if self._should_compute(key):
                self._memoize(key, func(*args, **kwargs))
            self._check_evict()
            return self._value(key)
        return inner

In [35]:
@Memoizer(cache_size=3)
def memfib2(n):
    if n < 2:
        return 1
    return memfib2(n - 1) + memfib2(n - 2)

In [36]:
memfib2(100)

573147844013817084101L

# Any Questions?
## https://github.com/csams/python_examples